In [109]:
import pandas as pd #Import pandas for operations
from sympy.printing.pytorch import torch

In [110]:
dataframe = pd.read_csv('data/sdss_100k_galaxy_form_burst.csv', low_memory=False, header=1)

print("Dataset loaded successfully!")
print(f"Dataset shape: {dataframe.shape}")
print(f"Columns: {list(dataframe.columns)}")
print(f"Datatypes: {dataframe.dtypes}")

Dataset loaded successfully!
Dataset shape: (100000, 43)
Columns: ['objid', 'specobjid', 'ra', 'dec', 'u', 'g', 'r', 'i', 'z', 'modelFlux_u', 'modelFlux_g', 'modelFlux_r', 'modelFlux_i', 'modelFlux_z', 'petroRad_u', 'petroRad_g', 'petroRad_i', 'petroRad_r', 'petroRad_z', 'petroFlux_u', 'petroFlux_g', 'petroFlux_i', 'petroFlux_r', 'petroFlux_z', 'petroR50_u', 'petroR50_g', 'petroR50_i', 'petroR50_r', 'petroR50_z', 'psfMag_u', 'psfMag_r', 'psfMag_g', 'psfMag_i', 'psfMag_z', 'expAB_u', 'expAB_g', 'expAB_r', 'expAB_i', 'expAB_z', 'class', 'subclass', 'redshift', 'redshift_err']
Datatypes: objid             int64
specobjid        uint64
ra              float64
dec             float64
u               float64
g               float64
r               float64
i               float64
z               float64
modelFlux_u     float64
modelFlux_g     float64
modelFlux_r     float64
modelFlux_i     float64
modelFlux_z     float64
petroRad_u      float64
petroRad_g      float64
petroRad_i      float64


In [111]:
# dataframe['subclass'].unique()


In [112]:
#1 Data pre-processing

#1.1.1 remove unnecessary columns
modified_dataframe = dataframe.drop(['objid', 'specobjid', 'class'], axis=1)

# #1.1.2 Encode subclass category accordingly
# modified_dataframe['subclass'] = modified_dataframe['subclass'].map({'STARBURST': 1, 'STARFORMING': 0})

#1.2.1 Split dataset into Test and Training data. (80% Training - 20% Test)
main_training_data = modified_dataframe.sample(frac=0.8)
testing_data = modified_dataframe.drop(main_training_data.index)

#1.2.2 Split Test data into Test and Validation Set. (70% Training - 30% Validation)
training_set = main_training_data.sample(frac=0.7)
validation_set = main_training_data.drop(training_set.index)


In [113]:
#Visualize data.
#1. Training Set Info
print(f"Training Dataset shape: {training_set.shape}")
# print(f"Training Dataset Columns: {list(training_set.columns)}")

#2. Testing Set Info
print(f"Testing Dataset shape: {testing_data.shape}")
# print(f"Testing Dataset Columns: {list(testing_data.columns)}")

#3. Validation Set Info
print(f"Validation Dataset shape: {validation_set.shape}")
# print(f"Validation Dataset Columns: {list(validation_set.columns)}")

assert(len(training_set) + len(testing_data) + len(validation_set) == len(modified_dataframe)) #Confirm that the lengths match.





Training Dataset shape: (56000, 40)
Testing Dataset shape: (20000, 40)
Validation Dataset shape: (24000, 40)


In [114]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

def plot_distributions_seaborn(df, figsize=(15, 12)):
    numerical_cols = df.select_dtypes(include=[np.number]).columns
    categorical_cols = df.select_dtypes(exclude=[np.number]).columns

    # Plot numerical distributions
    if len(numerical_cols) > 0:
        n_rows = (len(numerical_cols) + 2) // 3
        fig, axes = plt.subplots(n_rows, 3, figsize=figsize)
        axes = axes.flatten()

        for i, col in enumerate(numerical_cols):
            sns.histplot(data=df, x=col, ax=axes[i], kde=True, bins=30)
            axes[i].axvline(df[col].mean(), color='red', linestyle='--', alpha=0.8)
            axes[i].axvline(df[col].median(), color='green', linestyle='--', alpha=0.8)
            axes[i].set_title(f'Distribution of {col}')

        # Hide empty subplots
        for j in range(i + 1, len(axes)):
            axes[j].set_visible(False)

        plt.tight_layout()
        plt.show()

    # Plot categorical distributions
    if len(categorical_cols) > 0:
        n_rows = (len(categorical_cols) + 2) // 3
        fig, axes = plt.subplots(n_rows, 3, figsize=figsize)
        axes = axes.flatten()

        for i, col in enumerate(categorical_cols):
            value_counts = df[col].value_counts().head(10)
            sns.barplot(x=value_counts.index, y=value_counts.values, ax=axes[i])
            axes[i].set_title(f'Distribution of {col}')
            axes[i].tick_params(axis='x', rotation=45)
            axes[i].set_ylabel('Count')

        # Hide empty subplots
        for j in range(i + 1, len(axes)):
            axes[j].set_visible(False)

        plt.tight_layout()
        plt.show()

# plot_distributions_seaborn(training_set)

In [115]:
from sklearn.preprocessing import StandardScaler
#2. Normalize data.

#2. Apply mean-centering & variance scaling to data

#2.1. Define function
def standardize_with_sklearn(df):
    """
    Use scikit-learn's StandardScaler for robust standardization
    """
    # Separate numerical and categorical columns
    numerical_cols = df.select_dtypes(include=[np.number]).columns
    categorical_cols = df['subclass']

    # Create a copy and only scale numerical columns
    df_normalized = df.copy()

    if len(numerical_cols) > 0:
        scaler = StandardScaler()
        df_normalized[numerical_cols] = scaler.fit_transform(df[numerical_cols])
        df_normalized.drop('subclass', axis=1, inplace=True)

    return df_normalized, categorical_cols

#2.2. Apply standardization
testing_dataset_normalized, testing_dataset_labels = standardize_with_sklearn(testing_data)
training_dataset_normalized, training_dataset_labels = standardize_with_sklearn(training_set)
validation_dataset_normalized, validation_dataset_labels = standardize_with_sklearn(validation_set)

# testing_dataset_normalized.head(5)
# testing_dataset_normalized.shape
# testing_dataset_labels.shape
# validation_dataset_normalized.shape
# validation_dataset_labels.shape
#1.1.2 Encode subclass category accordingly
testing_dataset_labels = testing_dataset_labels.map({'STARBURST': 1, 'STARFORMING': 0})
training_dataset_labels = training_dataset_labels.map({'STARBURST': 1, 'STARFORMING': 0})
validation_dataset_labels = validation_dataset_labels.map({'STARBURST': 1, 'STARFORMING': 0})


In [116]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

#3. Begin Training Model.
# print(torch.cuda.is_available())

#3.1. Setup Model with Params. TODO: Specify weights!!
class galaxy_classification_nn(nn.Module):
    def __init__(self):

        super().__init__()
        self.input_layer = nn.Linear(in_features=39, out_features=133)
        self.hidden_layer = nn.Linear(in_features=133, out_features=133)
        self.output_layer = nn.Linear(in_features=133, out_features=2)

        #Specify activation functions.
        self.activation_function1 = nn.Tanh() # Input -> Hidden 1
        self.activation_function2 = nn.ReLU() # Hidden 1 -> Hidden 2
        self.activation_function3 = nn.Softmax(dim=1) # Hidden 2 -> Output TODO: Validate dim param

    def forward(self, x):
        x = self.input_layer(x)
        x = self.activation_function1(x)

        x = self.hidden_layer(x)
        x = self.activation_function2(x)

        x = self.output_layer(x)
        x = self.activation_function3(x)

        return x

        # x = F.relu(self.fc1(x)) # TODO: Check why not linear.
        # x = F.relu(self.fc2(x))
        # x = F.softmax(self.fc3(x), dim=1)
        # x = self.fc3(x)                  # raw scores (logits)
        return x


In [117]:
from torch.utils.data import TensorDataset, DataLoader


training_features = training_dataset_normalized.values

training_labels = training_dataset_labels.values

testing_features = testing_dataset_normalized.values

testing_labels = testing_dataset_labels.values


# TODO: Add Validation tensors
training_tensor_features = torch.tensor(training_features, dtype=torch.float32)

training_tensor_labels = torch.tensor(training_labels, dtype=torch.float32)

testing_tensor_features = torch.tensor(testing_features, dtype=torch.float32)

testing_tensor_labels = torch.tensor(testing_labels, dtype=torch.float32)

# Create Datasets
training_dataset = TensorDataset(training_tensor_features, training_tensor_labels)
testing_dataset = TensorDataset(testing_tensor_features, testing_tensor_labels)

training_dataset_loader = DataLoader(training_dataset, batch_size=64, shuffle=True)

testing_dataset_loader = DataLoader(testing_dataset, batch_size=64, shuffle=True)

print(training_tensor_features.dtype)
# print(training_dataset_labels)

# training_dataset_normalized.head(5)
#
# train_tensor_features = torch.tensor(training_dataset_normalized.values, dtype=torch.float32)
#
# train_tensor_labels = torch.tensor(training_dataset_labels.values, dtype=torch.float32)
#
# training_dataset = TensorDataset(train_tensor_features, train_tensor_labels)
#

torch.float32


In [118]:
import torch.optim as optim

#3.2. Init. model
device = torch.device("cuda:0" if torch.cuda.is_available () else "cpu")

galaxy_classification_model = galaxy_classification_nn().to(device)

#3.2. Setup Optimizer
loss_function = nn.CrossEntropyLoss()
error_optimizer = optim.SGD(galaxy_classification_model.parameters(), lr=0.01)


In [123]:
for epoch in range(5):
    print(f"Epoch: {epoch + 1}")
    # galaxy_classification_model.train()
    for  features, labels in training_dataset_loader:
        # print(f"Features: {features.shape}")
        # print(f"Targets: {targets.shape}")
        # print(f"Features DataType: {type(features)}")
        # print(f"Labels DataType: {type(targets)}")
        # print(features.shape, labels.shape)
        # print(labels)

        outputs = galaxy_classification_model(features)
        # print(f"Model outputs shape: {outputs.shape}")
    #     batch_X, batch_y = batch_X.to(device), batch_y.to(device)
    #
    #
    # error_optimizer.zero_grad()
    #
    # logits = galaxy_classification_model(batch_X)
    # loss = loss_function(logits, batch_y)
    # loss.backward()
    # error_optimizer.step()
    # print(f"Epoch {epoch+1} completed")



Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4
Epoch: 5
